# Import dependencies

In [1]:
import os
import sys

sys.path.insert(0, os.path.dirname(os.getcwd())) 

In [2]:
import time
import gc
import json

import numpy as np
import pandas as pd

from transformers import BertTokenizer, BertModel
from transformers import logging

from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score

import matplotlib.pyplot as plt

from InputDataset import InputDataset

import torch
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
from torch.nn.utils.rnn import pad_sequence

from torch import cuda

from ate_models.ATE_BERT_Dropout_CNN_BiLSTM_Linear import ATE_BERT_Dropout_CNN_BiLSTM_Linear

In [3]:
device = 'cuda' if cuda.is_available() else 'cpu'
logging.set_verbosity_error() 

In [4]:
print(torch.cuda.get_device_name(0))
print(f"Memory: {torch.cuda.get_device_properties(0).total_memory // 1024 ** 3} GB")

NVIDIA GeForce RTX 2060 SUPER
Memory: 8 GB


In [5]:
def clear_memory():
    torch.cuda.empty_cache()

    with torch.no_grad():
        torch.cuda.empty_cache()

    gc.collect()

# Load Data

In [6]:
DATASET = 'ATE_SemEval16_Restaurants_train.json'

In [7]:
df = pd.json_normalize(json.load(open(DATASET)))

In [8]:
df.head()

,text,tokens,iob_aspect_tags
0,Judging from previous posts this used to be a ...,"[Judging, from, previous, posts, this, used, t...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, ..."
1,"We, there were four of us, arrived at noon - t...","[We, ,, there, were, four, of, us, ,, arrived,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,"They never brought us complimentary noodles, i...","[They, never, brought, us, complimentary, nood...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,The food was lousy - too sweet or too salty an...,"[The, food, was, lousy, -, too, sweet, or, too...","[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0]"
4,The food was lousy - too sweet or too salty an...,"[The, food, was, lousy, -, too, sweet, or, too...","[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0]"


# Train & Validate

In [9]:
TRAIN_BATCH_SIZE = 4
VALID_BATCH_SIZE = 4

EPOCHS = 2

LEARNING_RATE = 1e-5

TRAIN_SPLIT = 0.8

NO_RUNS = 10

In [10]:
SEQ_LEN = 512

In [11]:
BERT_FINE_TUNED_PATH = '../../../results/ATE/SemEval16 - Task 5 - Restaurants/models/bert_fine_tuned_512.pth'

In [12]:
MODEL_OUTPUT = '../../../results/ATE/SemEval16 - Task 5 - Restaurants/models/bert_fine_tuned_dropout_cnn_bilstm_linear_512.pth'
STATS_OUTPUT = '../../../results/ATE/SemEval16 - Task 5 - Restaurants/stats/bert_fine_tuned_dropout_cnn_bilstm_linear_512.csv'

In [13]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [14]:
best_accuracy = 0.0

In [15]:
def create_mini_batch(samples):
    ids_tensors = [s[1] for s in samples]
    ids_tensors[0] = torch.nn.ConstantPad1d((0, SEQ_LEN - len(ids_tensors[0])), 0)(ids_tensors[0])
    ids_tensors = pad_sequence(ids_tensors, batch_first=True).to(device)

    tags_tensors = [s[2] for s in samples]
    tags_tensors[0] = torch.nn.ConstantPad1d((0, SEQ_LEN - len(tags_tensors[0])), 0)(tags_tensors[0])
    tags_tensors = pad_sequence(tags_tensors, batch_first=True).to(device)
    
    masks_tensors = torch.zeros(ids_tensors.shape, dtype=torch.long).to(device)
    masks_tensors = masks_tensors.masked_fill(ids_tensors != 0, 1).to(device)
    
    return ids_tensors, tags_tensors, masks_tensors

In [16]:
def train(epoch, model, loss_fn, optimizer, dataloader):
    model.train()

    dataloader_len = len(dataloader)

    losses = []

    for _,data in enumerate(dataloader, 0):
        optimizer.zero_grad()

        ids_tensors, tags_tensors, masks_tensors = data

        outputs = model(ids_tensors, masks_tensors)

        loss = loss_fn(outputs.view(-1, 3), tags_tensors.view(-1))

        losses.append(loss.item())
        
        if _ % (dataloader_len // 10) == 0:
            print(f"Epoch: {epoch}/{EPOCHS}, Batch: {_}/{dataloader_len}, Loss: {loss.item()}")
        
        loss.backward()
        
        optimizer.step()
    
    return losses

In [17]:
def validation(model, dataloader):
    model.eval()
    
    fin_targets=[]
    fin_outputs=[]

    with torch.no_grad():
        for _, data in enumerate(dataloader, 0):
            ids_tensors, tags_tensors, masks_tensors = data
            ids_tensors = ids_tensors.to(device)
            tags_tensors = tags_tensors.to(device)
            masks_tensors = masks_tensors.to(device)

            outputs = model(ids_tensors, masks_tensors)
            
            _, predictions = torch.max(outputs, dim=2)

            fin_outputs += list([int(p) for pred in predictions for p in pred])
            fin_targets += list([int(tag) for tags_tensor in tags_tensors for tag in tags_tensor])

    return fin_outputs, fin_targets

In [18]:
results = pd.DataFrame(columns=['accuracy','precision_score_micro','precision_score_macro','recall_score_micro','recall_score_macro','f1_score_micro','f1_score_macro', 'execution_time'])

In [19]:
for i in range(NO_RUNS):
    # clear cache cuda
    torch.cuda.empty_cache()
    with torch.no_grad():
        torch.cuda.empty_cache()
    gc.collect()

    start_time = time.time()

    print(f"Run {i + 1}/{NO_RUNS}")

    train_dataset = df.sample(frac=TRAIN_SPLIT)
    test_dataset = df.drop(train_dataset.index).reset_index(drop=True)
    train_dataset = train_dataset.reset_index(drop=True)

    training_set = InputDataset(train_dataset, tokenizer)
    testing_set = InputDataset(test_dataset, tokenizer)

    train_dataloader = DataLoader(
        training_set,
        sampler = RandomSampler(train_dataset),
        batch_size = TRAIN_BATCH_SIZE,
        drop_last = True,
        collate_fn=create_mini_batch
    )

    validation_dataloader = DataLoader(
        testing_set,
        sampler = SequentialSampler(testing_set),
        batch_size = VALID_BATCH_SIZE,
        drop_last = True,
        collate_fn=create_mini_batch
    )

    model = ATE_BERT_Dropout_CNN_BiLSTM_Linear(torch.load(BERT_FINE_TUNED_PATH), bert_seq_len=SEQ_LEN, dropout=0.3, bilstm_in_features=256, conv_out_channels=SEQ_LEN, conv_kernel_size=3, no_out_labels=3, device=device).to(device)

    optimizer = torch.optim.Adam(params = model.parameters(), lr=LEARNING_RATE)
    loss_fn = torch.nn.CrossEntropyLoss()

    train_losses = []
    for epoch in range(EPOCHS):
        losses = train(epoch, model, loss_fn, optimizer, train_dataloader)

        train_losses += losses

    outputs, targets = validation(model, validation_dataloader)

    plt.title(f'Train Loss for run {i + 1}/{NO_RUNS}')
    plt.plot(train_losses)
    plt.savefig(f'../../../results/ATE/SemEval16 - Task 5 - Restaurants/plots/bert_ft_do_cnn_bilstm_lin/train_loss_run_{i + 1}.png')

    plt.clf()
    
    accuracy = accuracy_score(targets, outputs)
    precision_score_micro = precision_score(targets, outputs, average='micro')
    precision_score_macro = precision_score(targets, outputs, average='macro')
    recall_score_micro = recall_score(targets, outputs, average='micro')
    recall_score_macro = recall_score(targets, outputs, average='macro')
    f1_score_micro = f1_score(targets, outputs, average='micro')
    f1_score_macro = f1_score(targets, outputs, average='macro')

    execution_time = time.time() - start_time

    results.loc[i] = [accuracy,precision_score_micro,precision_score_macro,recall_score_micro,recall_score_macro,f1_score_micro,f1_score_macro, execution_time]

    if accuracy > best_accuracy:
        best_accuracy = accuracy
        torch.save(model, MODEL_OUTPUT)

    del train_dataset
    del test_dataset
    del training_set
    del testing_set
    del model
    del loss_fn
    del optimizer
    del outputs
    del targets

Run 1/10
Epoch: 0/2, Batch: 0/501, Loss: 1.0976605415344238
Epoch: 0/2, Batch: 50/501, Loss: 0.6836255788803101
Epoch: 0/2, Batch: 100/501, Loss: 0.273844838142395
Epoch: 0/2, Batch: 150/501, Loss: 0.136262908577919
Epoch: 0/2, Batch: 200/501, Loss: 0.09060438722372055
Epoch: 0/2, Batch: 250/501, Loss: 0.07716764509677887
Epoch: 0/2, Batch: 300/501, Loss: 0.07023317366838455
Epoch: 0/2, Batch: 350/501, Loss: 0.06357183307409286
Epoch: 0/2, Batch: 400/501, Loss: 0.04533011093735695
Epoch: 0/2, Batch: 450/501, Loss: 0.05007601156830788
Epoch: 0/2, Batch: 500/501, Loss: 0.05163704231381416
Epoch: 1/2, Batch: 0/501, Loss: 0.04371737688779831
Epoch: 1/2, Batch: 50/501, Loss: 0.04019439220428467
Epoch: 1/2, Batch: 100/501, Loss: 0.03511502966284752
Epoch: 1/2, Batch: 150/501, Loss: 0.03373675420880318
Epoch: 1/2, Batch: 200/501, Loss: 0.029965728521347046
Epoch: 1/2, Batch: 250/501, Loss: 0.02951880916953087
Epoch: 1/2, Batch: 300/501, Loss: 0.03055153228342533
Epoch: 1/2, Batch: 350/501, Lo

d:\Apps\Anaconda\envs\ml\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Run 2/10
Epoch: 0/2, Batch: 0/501, Loss: 1.1350507736206055
Epoch: 0/2, Batch: 50/501, Loss: 0.4459030330181122
Epoch: 0/2, Batch: 100/501, Loss: 0.11481458693742752
Epoch: 0/2, Batch: 150/501, Loss: 0.06066460907459259
Epoch: 0/2, Batch: 200/501, Loss: 0.06281039863824844
Epoch: 0/2, Batch: 250/501, Loss: 0.028407763689756393
Epoch: 0/2, Batch: 300/501, Loss: 0.02652217075228691
Epoch: 0/2, Batch: 350/501, Loss: 0.02467682771384716
Epoch: 0/2, Batch: 400/501, Loss: 0.04064683988690376
Epoch: 0/2, Batch: 450/501, Loss: 0.016591107472777367
Epoch: 0/2, Batch: 500/501, Loss: 0.02181858941912651
Epoch: 1/2, Batch: 0/501, Loss: 0.022068694233894348
Epoch: 1/2, Batch: 50/501, Loss: 0.028483837842941284
Epoch: 1/2, Batch: 100/501, Loss: 0.013318856246769428
Epoch: 1/2, Batch: 150/501, Loss: 0.02533433400094509
Epoch: 1/2, Batch: 200/501, Loss: 0.0393470823764801
Epoch: 1/2, Batch: 250/501, Loss: 0.015824107453227043
Epoch: 1/2, Batch: 300/501, Loss: 0.010489302687346935
Epoch: 1/2, Batch: 35

d:\Apps\Anaconda\envs\ml\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Run 6/10
Epoch: 0/2, Batch: 0/501, Loss: 1.1332134008407593
Epoch: 0/2, Batch: 50/501, Loss: 0.5023646354675293
Epoch: 0/2, Batch: 100/501, Loss: 0.14383405447006226
Epoch: 0/2, Batch: 150/501, Loss: 0.06475847214460373
Epoch: 0/2, Batch: 200/501, Loss: 0.03894929587841034
Epoch: 0/2, Batch: 250/501, Loss: 0.03647876903414726
Epoch: 0/2, Batch: 300/501, Loss: 0.030374394729733467
Epoch: 0/2, Batch: 350/501, Loss: 0.044236261397600174
Epoch: 0/2, Batch: 400/501, Loss: 0.02048378810286522
Epoch: 0/2, Batch: 450/501, Loss: 0.01951773837208748
Epoch: 0/2, Batch: 500/501, Loss: 0.02915814146399498
Epoch: 1/2, Batch: 0/501, Loss: 0.017862048000097275
Epoch: 1/2, Batch: 50/501, Loss: 0.024895479902625084
Epoch: 1/2, Batch: 100/501, Loss: 0.024444056674838066
Epoch: 1/2, Batch: 150/501, Loss: 0.015969619154930115
Epoch: 1/2, Batch: 200/501, Loss: 0.032181400805711746
Epoch: 1/2, Batch: 250/501, Loss: 0.008789229206740856
Epoch: 1/2, Batch: 300/501, Loss: 0.04117729887366295
Epoch: 1/2, Batch: 

d:\Apps\Anaconda\envs\ml\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Run 7/10
Epoch: 0/2, Batch: 0/501, Loss: 1.144788384437561
Epoch: 0/2, Batch: 50/501, Loss: 0.39965352416038513
Epoch: 0/2, Batch: 100/501, Loss: 0.13667304813861847
Epoch: 0/2, Batch: 150/501, Loss: 0.06583153456449509
Epoch: 0/2, Batch: 200/501, Loss: 0.05569930374622345
Epoch: 0/2, Batch: 250/501, Loss: 0.05044631287455559
Epoch: 0/2, Batch: 300/501, Loss: 0.04838879778981209
Epoch: 0/2, Batch: 350/501, Loss: 0.02809957228600979
Epoch: 0/2, Batch: 400/501, Loss: 0.0217494685202837
Epoch: 0/2, Batch: 450/501, Loss: 0.02150377631187439
Epoch: 0/2, Batch: 500/501, Loss: 0.027784276753664017
Epoch: 1/2, Batch: 0/501, Loss: 0.028157738968729973
Epoch: 1/2, Batch: 50/501, Loss: 0.038965001702308655
Epoch: 1/2, Batch: 100/501, Loss: 0.0208070520311594
Epoch: 1/2, Batch: 150/501, Loss: 0.019859030842781067
Epoch: 1/2, Batch: 200/501, Loss: 0.0360417440533638
Epoch: 1/2, Batch: 250/501, Loss: 0.015347005799412727
Epoch: 1/2, Batch: 300/501, Loss: 0.009846854954957962
Epoch: 1/2, Batch: 350/5

d:\Apps\Anaconda\envs\ml\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Run 8/10
Epoch: 0/2, Batch: 0/501, Loss: 1.1093993186950684
Epoch: 0/2, Batch: 50/501, Loss: 0.42232051491737366
Epoch: 0/2, Batch: 100/501, Loss: 0.10673917829990387
Epoch: 0/2, Batch: 150/501, Loss: 0.060128431767225266
Epoch: 0/2, Batch: 200/501, Loss: 0.040338657796382904
Epoch: 0/2, Batch: 250/501, Loss: 0.030302824452519417
Epoch: 0/2, Batch: 300/501, Loss: 0.02971889078617096
Epoch: 0/2, Batch: 350/501, Loss: 0.03893318027257919
Epoch: 0/2, Batch: 400/501, Loss: 0.04369034618139267
Epoch: 0/2, Batch: 450/501, Loss: 0.016320399940013885
Epoch: 0/2, Batch: 500/501, Loss: 0.046475961804389954
Epoch: 1/2, Batch: 0/501, Loss: 0.028368595987558365
Epoch: 1/2, Batch: 50/501, Loss: 0.029555896297097206
Epoch: 1/2, Batch: 100/501, Loss: 0.02993716299533844
Epoch: 1/2, Batch: 150/501, Loss: 0.011598318815231323
Epoch: 1/2, Batch: 200/501, Loss: 0.030187392607331276
Epoch: 1/2, Batch: 250/501, Loss: 0.018613062798976898
Epoch: 1/2, Batch: 300/501, Loss: 0.016945140436291695
Epoch: 1/2, Bat

<Figure size 432x288 with 0 Axes>

In [20]:
results

,accuracy,precision_score_micro,precision_score_macro,recall_score_micro,recall_score_macro,f1_score_micro,f1_score_macro,execution_time
0,0.995219,0.995219,0.331740,0.995219,0.333333,0.995219,0.332535,378.079059
1,0.995211,0.995211,0.549511,0.995211,0.349132,0.995211,0.361231,372.331963
2,0.994895,0.994895,0.591409,0.994895,0.363667,0.994895,0.386250,369.907994
3,0.995016,0.995016,0.576180,0.995016,0.444683,0.995016,0.469607,370.942109
4,0.995578,0.995578,0.449635,0.995578,0.390058,0.995578,0.409226,369.574724
5,0.994988,0.994988,0.458146,0.994988,0.351945,0.994988,0.364967,369.602218
6,0.994938,0.994938,0.432715,0.994938,0.348859,0.994938,0.359446,370.868318
7,0.995367,0.995367,0.620380,0.995367,0.367451,0.995367,0.391520,367.454077
8,0.995145,0.995145,0.653608,0.995145,0.353087,0.995145,0.369785,367.983384
9,0.994598,0.994598,0.567176,0.994598,0.359050,0.994598,0.378929,361.873867


In [21]:
results.to_csv(STATS_OUTPUT)